In [ ]:
import os
import glob
import subprocess
import pandas as pd

# Filter sequences

In [ ]:
input="dom2-combined"
best_binders=pd.read_csv(f"output/{input}/af2_best.csv")
#all_binders=pd.read_csv(f"output/{input}/af2_results.csv")
best_binders

In [ ]:
#filtered = best_binders[(best_binders["rmsd"]<3)&(best_binders["binder-rmsd"]<4)&(best_binders["plddt"]>0.84)]
filtered = best_binders[(best_binders["plddt"]>0.9)&(best_binders["i_pae"]<7)&(best_binders["rmsd"]<3)]

filtered = filtered.sort_values(by='plddt', ascending=False)
filtered

# Prepare folder of best binders

In [ ]:
# Create output dir
binder_folder="input"
output_dir=f"output/{input}/opt_binders"
binders_input=f"{output_dir}/{binder_folder}"
os.makedirs(f"{binders_input}", exist_ok=True)

# Copy binder inputs to new dir
pdbs=filtered["model_path"].to_list()
for pdb in pdbs:
    !cp $pdb $binders_input

# Count .pdb files in the folder
pdb_files = glob.glob(os.path.join(binders_input, "*.pdb"))
pdb_count = len(pdb_files)
print(f"Number of binders to optimize: {pdb_count}. Will run {pdb_count} arrays")

# Run optimization for each binder in parallel

In [ ]:

# Binder optimization parameters
iterations=30
starting_mpnns=200

designs_per_iteration=10
mpnn_per_design=5
sort="i_pae"

af2_recycles=12
mpnn_sampling_temp=0.15

array_number=pdb_count
array_limit=10

# Script arguments
bash_arguments=f"--output=/dev/null --array=0-{array_number-1}%{array_limit}"
script_arguments=f"{binders_input} {output_dir} {iterations} {starting_mpnns} {designs_per_iteration} {mpnn_per_design} {sort} {af2_recycles} {mpnn_sampling_temp}"

command = f"sbatch {bash_arguments} helper_scripts/binder_opt.sh {script_arguments}"

print(f"command will run {array_number} binder optimizations, each doing")
print(f"{iterations} iterations with {designs_per_iteration*mpnn_per_design} sampled designs.")
print(f"Each script will return {iterations*designs_per_iteration*mpnn_per_design} designs ({iterations*designs_per_iteration*mpnn_per_design*array_number} in total)")
print(command)

In [ ]:
# Run the array bash script
subprocess.run(command, shell=True)

In [ ]:
!squeue --me